In [ ]:
import warnings

import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.exceptions import ConvergenceWarning
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler

# different learning rate schedules and momentum parameters
params = [
    {
        "solver": "sgd",
        "learning_rate": "constant",
        "momentum": 0,
        "learning_rate_init": 0.2,
    },
    {
        "solver": "sgd",
        "learning_rate": "constant",
        "momentum": 0.9,
        "nesterovs_momentum": False,
        "learning_rate_init": 0.2,
    },
    {
        "solver": "sgd",
        "learning_rate": "constant",
        "momentum": 0.9,
        "nesterovs_momentum": True,
        "learning_rate_init": 0.2,
    },
    {
        "solver": "sgd",
        "learning_rate": "invscaling",
        "momentum": 0,
        "learning_rate_init": 0.2,
    },
    {
        "solver": "sgd",
        "learning_rate": "invscaling",
        "momentum": 0.9,
        "nesterovs_momentum": False,
        "learning_rate_init": 0.2,
    },
    {
        "solver": "sgd",
        "learning_rate": "invscaling",
        "momentum": 0.9,
        "nesterovs_momentum": True,
        "learning_rate_init": 0.2,
    },
    {"solver": "adam", "learning_rate_init": 0.01},
]

labels = [
    # 각 설정에 대한 라벨
    "constant learning-rate",
    "constant with momentum",
    "constant with Nesterov's momentum",
    "inv-scaling learning-rate",
    "inv-scaling with momentum",
    "inv-scaling with Nesterov's momentum",
    "adam",
]

plot_args = [
    # 각 그래프에 대한 스타일 설정
    {"c": "red", "linestyle": "-"},
    {"c": "green", "linestyle": "-"},
    {"c": "blue", "linestyle": "-"},
    {"c": "red", "linestyle": "--"},
    {"c": "green", "linestyle": "--"},
    {"c": "blue", "linestyle": "--"},
    {"c": "black", "linestyle": "-"},
]

# 데이터셋별로 결과를 시각화하는 함수
def plot_on_dataset(X, y, ax, name):
    # 각 데이터셋에 대해, 모든 학습 전략에 대한 학습을 시각화한다
    print("\nlearning on dataset %s" % name)
    ax.set_title(name)

    # 데이터 전처리: 스케일링
    X = MinMaxScaler().fit_transform(X)
    mlps = []
    # 데이터셋의 크기에 따라 최대 반복 횟수를 설정
    if name == "digits":
        max_iter = 15  # 숫자 데이터셋은 크기가 크지만 빠르게 수렴한다
    else:
        max_iter = 400

    # 각 매개변수 설정에 대해 MLPClassifier를 훈련
    for label, param in zip(labels, params):
        print("training: %s" % label)
        mlp = MLPClassifier(random_state=0, max_iter=max_iter, **param)

        with warnings.catch_warnings():
            # 수렴 경고를 무시
            warnings.filterwarnings(
                "ignore", category=ConvergenceWarning, module="sklearn"
            )
            mlp.fit(X, y)

        mlps.append(mlp)
        print("Training set score: %f" % mlp.score(X, y))
        print("Training set loss: %f" % mlp.loss_)
    # 각 MLP 모델의 손실 곡선을 그린다
    for mlp, label, args in zip(mlps, labels, plot_args):
        ax.plot(mlp.loss_curve_, label=label, **args)


# 2x2 서브플롯에서 각각의 데이터셋에 대한 학습 곡선을 시각화
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
# 네 가지 데이터셋을 로드
iris = datasets.load_iris()
X_digits, y_digits = datasets.load_digits(return_X_y=True)
data_sets = [
    (iris.data, iris.target),
    (X_digits, y_digits),
    datasets.make_circles(noise=0.2, factor=0.5, random_state=1),
    datasets.make_moons(noise=0.3, random_state=0),
]

# 각 데이터셋에 대해 시각화 실행
for ax, data, name in zip(
    axes.ravel(), data_sets, ["iris", "digits", "circles", "moons"]
):
    plot_on_dataset(*data, ax=ax, name=name)

# 범례를 추가하고 그래프를 표시
fig.legend(ax.get_lines(), labels, ncol=3, loc="upper center")
plt.show()